In [1]:
from pandas import DataFrame, read_csv
import pandas as pd
import random
import numpy as np
import json
from collections import defaultdict, Counter
import math
import sys
import itertools
import time
import scipy.stats 
from sklearn.neighbors import KernelDensity
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re
from scipy.optimize import curve_fit
import copy
import dill
from sklearn.neural_network import MLPClassifier


chr_lengths= {1: 643000,
                         2: 947000,
                         3: 1100000,
                         4: 1200000,
                         5: 1350000,
                         6: 1420000,
                         7: 1450000,
                         8: 1500000,
                         9: 1550000,
                         10: 1700000,
                         11: 2049999,
                         12: 2300000,
                         13: 2950000,
                         14: 3300000}

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """

    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):  #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

cpalette1 = sns.color_palette('Reds_r', 2)
cpalette2 = sns.color_palette('Blues_r', 3)
cpalette3 = sns.color_palette('Greys_r', 3)

color_map_dict = {'PC': cpalette1[0],
                 'FS': cpalette1[1],
                 'MS': cpalette1[1],
                 'GC': cpalette2[0],
                 'HS': cpalette2[1],
                  'FAV': cpalette2[2],
                  'GGC': cpalette3[0],
                  'HAV': cpalette3[1],
                  'FCS': cpalette3[2]}

In [2]:
def combine_results(pattern, metric):
    if metric != 'r_totals':
        combined_dict = defaultdict(lambda: defaultdict(list))
    else:
        combined_dict = defaultdict(list)
    for file in glob.glob(pattern):
        file_dict = json.load(open(file))
        print(file)
        for relationship in file_dict:
            if metric != 'r_totals':
                for chrom in file_dict[relationship]:
                    for segment_lengths in file_dict[relationship][chrom]:
                        combined_dict[relationship][chrom].append(segment_lengths)
            else:
                combined_dict[relationship] += file_dict[relationship]
    return combined_dict


def combine_sims(directory):
    fs_file_patterns = ['fs*_ibd_segment_numbers.json', 'fs*_ibd_segment_max.json',
               'fs*_r_totals.json']
    combined_dicts = {}
    for pattern in fs_file_patterns:
        match = re.search('fs\*_(.+).json',pattern)
        metric = match.groups()[0]
        combined_dicts[metric] = combine_results(directory + pattern, metric)
        json.dump(combined_dicts[metric], open(directory + 'fs_{m}_combined.json'.format(m=metric), 'w'), cls = NumpyEncoder)
    
    ms_file_patterns = ['ms*_ibd_segment_numbers.json', 'ms*_ibd_segment_max.json',
               'ms*_r_totals.json']

    ms_combined_dicts = {}
    for pattern in ms_file_patterns:
        match = re.search('ms\*_(.+).json',pattern)
        metric = match.groups()[0]
        ms_combined_dicts[metric] = combine_results(directory + pattern, metric)
        json.dump(ms_combined_dicts[metric], open(directory + 'ms_{m}_combined.json'.format(m=metric), 'w'), cls = NumpyEncoder)

combine_sims('2.0_11.3/')

2.0_11.3/fs9_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs4_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs6_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs8_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs5_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs7_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs1_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs3_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs10_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs2_2.0_11.3_ibd_segment_numbers.json
2.0_11.3/fs4_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs3_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs10_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs5_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs2_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs1_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs9_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs6_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs7_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs8_2.0_11.3_ibd_segment_max.json
2.0_11.3/fs7_2.0_11.3_r_totals.json
2.0_11.3/fs3_2.0_11.3_r_totals.json
2.0_11.3/fs9_2.0_11.3_r_to

In [3]:
param_dict = {}

for param_combo in ['2.0_11.3']:
    sim_types = ['fs', 'ms']
    raw_sim_data = defaultdict(dict)
    for sim_type in sim_types:    
        if sim_type == 'fs':
            raw_sim_data[sim_type]['r_totals'] = json.load(open('{p}/fs_r_totals_combined.json'.format(p=param_combo)))
            raw_sim_data[sim_type]['ibd_segment_max'] = json.load(open('{p}/fs_ibd_segment_max_combined.json'.format(p=param_combo)))
            raw_sim_data[sim_type]['ibd_segment_numbers'] = json.load(open('{p}/fs_ibd_segment_numbers_combined.json'.format(p=param_combo)))
        else:
            raw_sim_data[sim_type]['r_totals'] = json.load(open('{p}/ms_r_totals_combined.json'.format(p=param_combo)))
            raw_sim_data[sim_type]['ibd_segment_max'] = json.load(open('{p}/ms_ibd_segment_max_combined.json'.format(p=param_combo)))
            raw_sim_data[sim_type]['ibd_segment_numbers'] = json.load(open('{p}/ms_ibd_segment_numbers_combined.json'.format(p=param_combo)))
        max_segment_p_dict = defaultdict(dict)
        for relationship in raw_sim_data[sim_type]['ibd_segment_max']:
            for chrom in raw_sim_data[sim_type]['ibd_segment_max'][relationship]:
                max_segment_p_dict[relationship][chrom] = np.asarray([x for x in np.asarray(raw_sim_data[sim_type]['ibd_segment_max'][relationship][chrom]) / chr_lengths[int(chrom)]])
        raw_sim_data[sim_type]['p_ibd_max'] = max_segment_p_dict
    param_dict[param_combo] = copy.deepcopy(raw_sim_data)



In [4]:
def evaluate_max_segment_piecewise_pdf(x, relationship, chrom, pdf_dict, bandwidth =0.02):
    '''modeled as a kde with spikes'''
    #print(fs_max_segment_pdf_dict[relationship][int(chrom)])
    p0,p1,kde, tstep, = pdf_dict[relationship][int(chrom)]
                       
    if float(x) <= 0 + bandwidth:
        return p0/tstep
    elif float(x) >= 1 - bandwidth:
        return p1/tstep
    else:
        x_transmute = np.asarray([x])
        #if beta_dis:
        #    pdf = scipy.stats.beta.pdf(x, alpha, beta, loc, scale)
        #else:
        pdf = np.exp(kde.score_samples(x_transmute.reshape(-1,1)))[0]
        pdf = pdf * (1-p0-p1)
        return pdf

In [9]:
def fit_beta(r_dict):
    beta_variables = {}
    for relationship in r_dict:
        r_list = r_dict[relationship]
        r_list = [x if x != 0 else 1e-9 for x in r_list]
        alpha, beta, loc, scale = scipy.stats.beta.fit(r_list, floc=0,fscale=1)
        #x = np.linspace(0, 1, 100)
        beta_variables[relationship] = (alpha, beta, loc, scale)
        
    return beta_variables

def create_pdfs(p_max_segment_dict, bandwidth= 0.02):
    max_segment_pdf_dict = defaultdict(dict)
    for relationship in p_max_segment_dict:
        for chromosome in p_max_segment_dict[relationship]:
            data = np.asarray(p_max_segment_dict[relationship][chromosome])
            mask0 = np.asarray(p_max_segment_dict[relationship][chromosome]) <= 0 + bandwidth
            mask1 = np.asarray(p_max_segment_dict[relationship][chromosome]) >= 1 - bandwidth
            mask = ~mask0 & ~mask1
            data = np.asarray(data[mask])

            p0 = (np.sum(mask0) + 1 )/ (len(mask0) + 3)
            p1 = (np.sum(mask1) + 1 )/ (len(mask1) + 3)
            kde = KernelDensity(kernel='gaussian',bandwidth=bandwidth).fit(data.reshape(-1,1))#training of model
            #beta_params = scipy.stats.beta.fit(data)

            max_segment_pdf_dict[relationship][int(chromosome)] =  (p0, p1, kde, bandwidth)#, beta_params)
    return max_segment_pdf_dict

def calc_seg_count_pmf(segment_counts_dict):
    '''empirical segment_count_pmf, with add one smoothing
    theta_i = (x_i + alpha) / (N + alpha * d)
    where x_i is the count of the cateogory i
    alpha is set to 1
    d is the number of categories
    N is the sample count'''
    seg_count_pmf = defaultdict(lambda: defaultdict(lambda : defaultdict(dict)))
    for relationship in segment_counts_dict:
        for chrom in segment_counts_dict[relationship]:
            counts = Counter(segment_counts_dict[relationship][chrom])
            total = np.sum(list(counts.values()))
            max_count= max(counts)
            bins = range(0,max_count + 2) #add an additional category representing max + 1
            n_bins = len(bins)
            for x in bins:
                if x in counts:
                    seg_count_pmf[relationship][chrom][x] = (counts[x] + 1)/(total + n_bins)
                else:
                    seg_count_pmf[relationship][chrom]['misc'] = 1 / (total + n_bins)
    return seg_count_pmf

def evaluate_max_segment_piecewise_pdf(x, relationship, chrom, pdf_dict, bandwidth =0.02):
    '''modeled as a kde with spikes'''
    #print(fs_max_segment_pdf_dict[relationship][int(chrom)])
    p0,p1,kde, tstep, = pdf_dict[relationship][int(chrom)]
                       
    if float(x) <= 0 + bandwidth:
        return p0/tstep
    elif float(x) >= 1 - bandwidth:
        return p1/tstep
    else:
        x_transmute = np.asarray([x])
        #if beta_dis:
        #    pdf = scipy.stats.beta.pdf(x, alpha, beta, loc, scale)
        #else:
        pdf = np.exp(kde.score_samples(x_transmute.reshape(-1,1)))[0]
        pdf = pdf * (1-p0-p1)
        return pdf


    
def create_joint_kde(r_dict,l_dict, n_dict):
    kde_dict = {}
    for G in r_dict:
        consolidated_data = DataFrame()
        consolidated_data['r_total'] = r_dict[G]
        for chrom in range(1,15):
            consolidated_data['max_ibd_{x}'.format(x=chrom)] = l_dict[G][str(chrom)]
            consolidated_data['n_segment_{x}'.format(x=chrom)] = n_dict[G][str(chrom)]

        data = np.asarray(consolidated_data)[:4000]
        kde = KernelDensity(kernel='gaussian', bandwidth = 0.01).fit(data)
        kde_dict[G] = kde
    return kde_dict


    
raw_sim_data = param_dict['2.0_11.3']
tmp_pdfs = defaultdict(dict)
for sim_type in raw_sim_data:
    tmp_pdfs[sim_type]['r_beta'] = fit_beta(raw_sim_data[sim_type]['r_totals'])
    tmp_pdfs[sim_type]['p_max_segment'] = create_pdfs(raw_sim_data[sim_type]['p_ibd_max'])
    tmp_pdfs[sim_type]['segment_count'] = calc_seg_count_pmf(raw_sim_data[sim_type]['ibd_segment_numbers'])
    tmp_pdfs[sim_type]['joint'] = create_joint_kde(raw_sim_data[sim_type]['r_totals'],
                                                 raw_sim_data[sim_type]['p_ibd_max'],
                                                 raw_sim_data[sim_type]['ibd_segment_numbers'])
    
pdfs = {}
pdfs['r_beta'] = tmp_pdfs['fs']['r_beta']
pdfs['p_max_segment'] = tmp_pdfs['fs']['p_max_segment']
pdfs['segment_count'] = tmp_pdfs['fs']['segment_count']

for key in pdfs:
    pdfs[key].pop('MS')

for G in ['MS', 'FAV', 'FCS']: #subset of relationships that involve the MS and are thus affected
    pdfs['r_beta'][G + '.MS'] = tmp_pdfs['ms']['r_beta'][G]
    pdfs['p_max_segment'][G + '.MS'] = tmp_pdfs['ms']['p_max_segment'][G]
    pdfs['segment_count'][G + '.MS'] = tmp_pdfs['ms']['segment_count'][G]

In [10]:
with open('sim_parameters.pkl', 'wb') as fp:
    dill.dump(pdfs, fp)

In [188]:
class Sim:
    def __init__(self, relationship, r_total, max_ibd_segment, n_segment_count):
        self.relationship = relationship
        self.r_total = r_total

        self.max_ibd_segment = max_ibd_segment
        self.n_segment_count = n_segment_count
        self.calc_all_likelihoods()
    
    def likelihood(self, G, r_flag = 1, ibdmax_flag = 1, count_flag = 1):
        r_total_beta_params = pdfs['r_beta'][G]
        logL = 0
        P_rtotal = scipy.stats.beta.logpdf(self.r_total, *r_total_beta_params)
        P_ibdmax = 0
        P_seg_count = 0
        for chrom in range(1,15):
            idx = chrom - 1
            P_ibdmax += np.log(evaluate_max_segment_piecewise_pdf(self.max_ibd_segment[idx], G, chrom, \
                                                            pdfs['p_max_segment']))

            n_segments = self.n_segment_count[idx]
            if n_segments in pdfs['segment_count'][G][str(chrom)].keys():
                P_seg_count += np.log(pdfs['segment_count'][G][str(chrom)][n_segments])
            else:
                P_seg_count += np.log(pdfs['segment_count'][G][str(chrom)]['misc'])
        logL = P_rtotal * r_flag +  P_ibdmax * ibdmax_flag + P_seg_count * count_flag

        return logL
    
    def calc_all_likelihoods(self, r_flag = 1, ibdmax_flag = 1, count_flag = 1):
        self.complete_likelihoods = {}
        self.r_likelihoods = {}
        for G in ['PC', 'GC', 'GGC', 'FS', 'HS', 'FAV', 'HAV', 'FCS', 'MS.MS', 'FAV.MS', 'FCS.MS']:
            self.complete_likelihoods[G] = self.likelihood(G, r_flag, ibdmax_flag, count_flag)
            self.r_likelihoods[G] = self.likelihood(G, 1, 0, 0)
        
        self.max_complete_likelihood = max(self.complete_likelihoods, key=self.complete_likelihoods.get)
        self.max_r_likelihood = max(self.r_likelihoods, key=self.r_likelihoods.get)
        
        
            

        

n_sims = len(raw_sim_data['fs']['ibd_segment_numbers']['PC']['1'])
sim_indiv = defaultdict(list)
sim_type = 'fs'
for relationship in ['PC', 'GC', 'GGC', 'FS', 'HS', 'FAV', 'HAV', 'FCS']:
    print(relationship)
    for sim_idx in range(n_sims):
        if sim_idx % 500 == 0:
            print(sim_idx)
        max_segment_list = [raw_sim_data[sim_type]['p_ibd_max'][relationship][str(chrom)][sim_idx] for chrom in range(1,15)]
        count_list = [raw_sim_data[sim_type]['ibd_segment_numbers'][relationship][str(chrom)][sim_idx] for chrom in range(1,15)]
        
        sim_indiv[relationship].append(Sim(relationship, raw_sim_data[sim_type]['r_totals'][relationship][sim_idx], \
                                      max_segment_list, count_list))
        
sim_type = 'ms'
for relationship in ['MS.MS', 'FAV.MS', 'FCS.MS']:
    print(relationship)
    for sim_idx in range(n_sims):
        if sim_idx % 500 == 0:
            print(sim_idx)
        base_r = relationship.split('.')[0]
        max_segment_list = [raw_sim_data[sim_type]['p_ibd_max'][base_r][str(chrom)][sim_idx] for chrom in range(1,15)]
        count_list = [raw_sim_data[sim_type]['ibd_segment_numbers'][base_r][str(chrom)][sim_idx] for chrom in range(1,15)]
        
        sim_indiv[relationship].append(Sim(relationship, raw_sim_data[sim_type]['r_totals'][base_r][sim_idx], \
                                      max_segment_list, count_list))

PC
0
500
1000
1500
2000
2500
3000
3500
4000
4500
GC
0
500
1000
1500
2000
2500
3000
3500
4000
4500
GGC
0
500
1000
1500
2000
2500
3000
3500
4000
4500
FS
0
500
1000
1500
2000
2500
3000
3500
4000
4500
HS
0
500
1000
1500
2000
2500
3000
3500
4000
4500
FAV
0
500
1000
1500
2000
2500
3000
3500
4000
4500
HAV
0
500
1000
1500
2000
2500
3000
3500
4000
4500
FCS
0
500
1000
1500
2000
2500
3000
3500
4000
4500
MS.MS
0
500
1000
1500
2000
2500
3000
3500
4000
4500
FAV.MS
0
500
1000
1500
2000
2500
3000
3500
4000
4500
FCS.MS
0
500
1000
1500
2000
2500
3000
3500
4000
4500
